In [37]:
from openai import OpenAI
import json
client = OpenAI()

Upload the training and validation file

In [2]:
client.files.create(
  file=open("data/training_examples.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-ScyLiRDbsRADrD5yYnEH1TIH', bytes=2933367, created_at=1713461900, filename='training_examples.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [3]:
client.files.create(
  file=open("data/validation.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-wPiN8afeyXCVrsKtqguiinLo', bytes=793405, created_at=1713461904, filename='validation.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [5]:
""" client.fine_tuning.jobs.create(
  training_file="file-ScyLiRDbsRADrD5yYnEH1TIH",
  validation_file="file-wPiN8afeyXCVrsKtqguiinLo",
  model="gpt-3.5-turbo"
) """

FineTuningJob(id='ftjob-5MD2SBEzsv7xZcBI47hmvhYe', created_at=1713461944, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-HzgmxGLTS1k6jx5kb4K1RDPE', result_files=[], seed=1529875545, status='validating_files', trained_tokens=None, training_file='file-ScyLiRDbsRADrD5yYnEH1TIH', validation_file='file-wPiN8afeyXCVrsKtqguiinLo', integrations=[], user_provided_suffix=None)

In [21]:
# List 10 fine-tuning jobs
jobs = client.fine_tuning.jobs.list(limit=10)

In [22]:
def print_job_details(jobs):
    for job in jobs.data:
        print("Job ID:", job.id)
        print("Model:", job.model)
        print("Fine-Tuned Model:", getattr(job, 'fine_tuned_model', 'N/A'))
        print("Status:", job.status)
        print("Created At:", job.created_at)
        print("Finished At:", getattr(job, 'finished_at', 'N/A'))
        print("Trained Tokens:", getattr(job, 'trained_tokens', 'N/A'))
        print("Error Message:", job.error.message if job.error and job.error.message else "No error")
        print("Training File:", job.training_file)
        print("Validation File:", getattr(job, 'validation_file', 'N/A'))
        print("Result Files:", job.result_files)
        print("Hyperparameters:", {key: getattr(job.hyperparameters, key) for key in job.hyperparameters.__annotations__})
        print("-" * 40)

# Print the formatted job details
print_job_details(jobs)

Job ID: ftjob-5MD2SBEzsv7xZcBI47hmvhYe
Model: gpt-3.5-turbo-0125
Fine-Tuned Model: ft:gpt-3.5-turbo-0125:personal::9FR6HhPm
Status: succeeded
Created At: 1713461944
Finished At: 1713466120
Trained Tokens: 1458567
Error Message: No error
Training File: file-ScyLiRDbsRADrD5yYnEH1TIH
Validation File: file-wPiN8afeyXCVrsKtqguiinLo
Result Files: ['file-yhFb55sMYfMMMzzW0f2djkXD']
Hyperparameters: {'n_epochs': 3}
----------------------------------------
Job ID: ftjob-IezDF9XKsGU67cAvy4kIEg6N
Model: gpt-3.5-turbo-0125
Fine-Tuned Model: ft:gpt-3.5-turbo-0125:personal::8xou7HV2
Status: succeeded
Created At: 1709264327
Finished At: 1709267718
Trained Tokens: 70329
Error Message: No error
Training File: file-yjr0iziZqYWZ0RQRnT82i1zy
Validation File: None
Result Files: ['file-4i5Fb6UJrBBzSBlIyoUPaWA2']
Hyperparameters: {'n_epochs': 3}
----------------------------------------
Job ID: ftjob-cdjtCSRVA1XewcEF5dNw3AMk
Model: gpt-3.5-turbo-0613
Fine-Tuned Model: ft:gpt-3.5-turbo-0613:personal::8RoHogRu
S

In [24]:

# Retrieve the state of a fine-tune
fine_tune_state = client.fine_tuning.jobs.retrieve("ftjob-5MD2SBEzsv7xZcBI47hmvhYe")

In [27]:
# Function to print the state of a fine-tune
def print_job_details(fine_tune_state ):
    print("Job ID:", fine_tune_state .id)
    print("Model:", fine_tune_state .model)
    print("Fine-Tuned Model:", fine_tune_state .fine_tuned_model)
    print("Status:", fine_tune_state .status)
    print("Created At:", fine_tune_state .created_at)
    print("Finished At:", fine_tune_state .finished_at)
    print("Trained Tokens:", fine_tune_state .trained_tokens)
    #print("Error Message:", fine_tune_state .error['message'] if fine_tune_state .error['message'] else "No error")
    print("Training File:", fine_tune_state .training_file)
    print("Validation File:", fine_tune_state .validation_file)
    print("Result Files:", fine_tune_state .result_files)
    print("Hyperparameters:", fine_tune_state .hyperparameters)
    print("-" * 40)

# Print the formatted state of a fine-tune
print_job_details(fine_tune_state )

Job ID: ftjob-5MD2SBEzsv7xZcBI47hmvhYe
Model: gpt-3.5-turbo-0125
Fine-Tuned Model: ft:gpt-3.5-turbo-0125:personal::9FR6HhPm
Status: succeeded
Created At: 1713461944
Finished At: 1713466120
Trained Tokens: 1458567
Training File: file-ScyLiRDbsRADrD5yYnEH1TIH
Validation File: file-wPiN8afeyXCVrsKtqguiinLo
Result Files: ['file-yhFb55sMYfMMMzzW0f2djkXD']
Hyperparameters: Hyperparameters(n_epochs=3, batch_size=2, learning_rate_multiplier=2)
----------------------------------------


In [36]:
# Define the FineTuningJobEvent class
class FineTuningJobEvent:
    def __init__(self, id, created_at, level, message, object, data, type):
        self.id = id
        self.created_at = created_at
        self.level = level
        self.message = message
        self.object = object
        self.data = data
        self.type = type

# Define the SyncCursorPage class
class SyncCursorPage:
    def __init__(self, data, object, has_more):
        self.data = data
        self.object = object
        self.has_more = has_more

# Function to print job event details
def print_job_event_details(events_page):
    print("Object Type:", events_page.object)
    print("Has More Events:", events_page.has_more)
    print("-" * 40)
    for event in events_page.data:
        print("Event ID:", event.id)
        print("Created At:", event.created_at)
        print("Level:", event.level)
        print("Message:", event.message)
        if event.data:
            print("Additional Data:")
            for key, value in event.data.items():
                print(f"  {key}: {value}")
        print("-" * 40)

# Fetch up to 10 events from a fine-tuning job
events_page = client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-5MD2SBEzsv7xZcBI47hmvhYe", limit=10)

# Convert the data to FineTuningJobEvent objects (replace this with actual data conversion)
events = []
for event_data in events_page.data:
    event = FineTuningJobEvent(
        id=event_data.id,
        created_at=event_data.created_at,
        level=event_data.level,
        message=event_data.message,
        object=event_data.object,
        data=event_data.data,
        type=event_data.type
    )
    events.append(event)

# Create a SyncCursorPage object (replace this with the actual object received from the API)
events_page = SyncCursorPage(events, 'list', has_more=False)

# Print the formatted job event details
print_job_event_details(events_page)

Object Type: list
Has More Events: False
----------------------------------------
Event ID: ftevent-I9dMgDtiJIrLA4TtvEiqErnL
Created At: 1713466125
Level: info
Message: The job has successfully completed
----------------------------------------
Event ID: ftevent-lX9SgVjxHiteNr1FGv5wzNet
Created At: 1713466122
Level: info
Message: New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal::9FR6HhPm
----------------------------------------
Event ID: ftevent-CBSqlUcKD31PnO20NjCqRkXu
Created At: 1713466122
Level: info
Message: Checkpoint created at step 1110 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9FR6HTsJ:ckpt-step-1110
----------------------------------------
Event ID: ftevent-esamAgfr0dMJrkVtiQelGCkk
Created At: 1713466122
Level: info
Message: Checkpoint created at step 555 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9FR6HUC4:ckpt-step-555
----------------------------------------
Event ID: ftevent-nIkX6hjlp6UU3cWnYWLb62NK
Created At: 1713466117
Level: info
Message: Ste

In [ ]:
""" 
# Cancel a job
client.fine_tuning.jobs.cancel("ftjob-abc123") """


""" # Delete a fine-tuned model (must be an owner of the org the model was created in)
client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123") """

In [ ]:
""" curl -X POST \\
    -H "Content-Type: application/json" \\
    -H "Authorization: Bearer $OPENAI_API_KEY" \\
    -d '{
    "model": "gpt-3.5-turbo-0125",
    "training_file": "file-ScyLiRDbsRADrD5yYnEH1TIH",
    "validation_file": "file-wPiN8afeyXCVrsKtqguiinLo",
    "integrations": [
        {
            "type": "wandb",
            "wandb": {
                "project": "patent-novelty",
                "tags": ["project:tag", "lineage"]
            }
        }
    ]
}' https://api.openai.com/v1/fine_tuning/jobs """

In [ ]:
https://wandb.ai/<WANDB-ENTITY>/patent-novelty/runs/ftjob-fb2AtaV0eUh15MAb0bMqUzgr